# Baseline NL→SQL Evaluation (Zero-shot vs Few-shot)

This notebook is the baseline replication block for Ojuri et al. (2025): it measures prompting effects (`k=0` vs `k=3`) on the same 200-item ClassicModels test set using an open-source local stack.

Primary role in dissertation:
- establish non-fine-tuned reference metrics,
- isolate prompting gains before QLoRA,
- provide controlled inputs for later paired comparisons.

This notebook runs the VA/EX baseline over `data/classicmodels_test_200.json` and saves outputs under `results/baseline/`.


Run this setup in a fresh GPU runtime, then restart before continuing.

In [ ]:
%%bash
set -e
export PIP_DEFAULT_TIMEOUT=120

# Clean conflicting preinstalls
pip uninstall -y torch torchvision torchaudio bitsandbytes triton transformers accelerate peft trl datasets numpy pandas fsspec requests google-auth scipy scikit-learn || true

# Base deps
pip install -q --no-cache-dir --force-reinstall \
  numpy==1.26.4 pandas==2.2.1 scipy scikit-learn \
  fsspec==2024.5.0 requests==2.31.0 google-auth==2.43.0

# Torch + CUDA 12.1
pip install -q --no-cache-dir --force-reinstall \
  torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 \
  --index-url https://download.pytorch.org/whl/cu121

# bitsandbytes + triton + HF stack
pip install -q --no-cache-dir --force-reinstall \
  bitsandbytes==0.43.3 triton==2.3.1 \
  transformers==4.44.2 accelerate==0.33.0 peft==0.17.0 trl==0.9.6 datasets==2.20.0

echo "Setup complete. Restart runtime once, then run the rest of the notebook top-to-bottom."


After restart, continue with DB/auth, schema, model, and eval cells.

Prompt/eval: build prompts (system+schema+k exemplars), generate SQL, postprocess, and compute VA/EX/EM.

Practical note: we keep schema context explicit so generated SQL is grounded in real tables and columns.


In [ ]:
import os, sys, shutil
from pathlib import Path

# If this notebook is opened directly in Colab (not from a cloned repo), clone the repo first.
if Path("data/classicmodels_test_200.json").exists() is False and Path("/content").exists():
    repo_dir = Path("/content/NLtoSQL")
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    !git clone https://github.com/MacKenzieOBrian/NLtoSQL.git "{repo_dir}"
    os.chdir(repo_dir)

# Ensure repo root is on sys.path for `import nl2sql`
sys.path.insert(0, os.getcwd())
print("cwd:", os.getcwd())


## Install dependencies (Colab)

This repo pins versions in `requirements.txt` to reduce Colab binary drift.
After installation, restart the runtime (Runtime → Restart runtime), then run this notebook again from the top.


In [ ]:
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -r requirements.txt
else:
    print("Not in Colab; ensure requirements are installed.")


In [ ]:
# Colab-only: authenticate with GCP (safe to skip locally)
try:
    from google.colab import auth
except ModuleNotFoundError:
    auth = None

if auth:
    auth.authenticate_user()
else:
    print("Not running in Colab; ensure ADC or service account auth is configured.")


In [ ]:
# Hugging Face auth (gated model)
import os

hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
if hf_token:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
    print("Using HF token from env")
else:
    try:
        from huggingface_hub import notebook_login
        notebook_login()
    except Exception as e:
        print("HF auth not configured:", e)


Environment note: DB access is via Cloud SQL connector + SQLAlchemy creator hook through `nl2sql.db`.


In [ ]:
import json
from getpass import getpass

INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME", "classicmodels")

if not INSTANCE_CONNECTION_NAME:
    INSTANCE_CONNECTION_NAME = input("Enter INSTANCE_CONNECTION_NAME: ").strip()
if not DB_USER:
    DB_USER = input("Enter DB_USER: ").strip()
if not DB_PASS:
    DB_PASS = getpass("Enter DB_PASS: ")

print("Using DB:", DB_NAME)

test_set = json.loads(open("data/classicmodels_test_200.json", "r", encoding="utf-8").read())
print("Loaded test items:", len(test_set))


Data note: the benchmark stays fixed at 200 items so comparisons remain paired across runs.


In [ ]:
from nl2sql.db import create_engine_with_connector

engine, connector = create_engine_with_connector(
    instance_connection_name=INSTANCE_CONNECTION_NAME,
    user=DB_USER,
    password=DB_PASS,
    db_name=DB_NAME,
)

print("Engine ready")


Model note: default loading uses 4-bit quantization to fit common Colab GPU memory.


Fallback note: if 4-bit load fails, use the built-in fallback path in the next cell (fp16/bf16).


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"

print("Loading tokenizer...")
tok = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Try 4-bit loading (fallback to fp/bf16)
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    print("Attempting 4-bit quantized load...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        token=True,
    )
except Exception as e:
    print("4-bit load failed, falling back. Error:")
    print(e)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        token=True,
    )

model.generation_config.do_sample = False
model.generation_config.num_beams = 1

print("Model device:", model.device)


Schema note: `build_schema_summary(...)` generates prompt context from live DB metadata.


Consistency note: keep prompt and schema settings unchanged when making cross-run claims.


In [ ]:
from nl2sql.schema import build_schema_summary

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME, max_cols_per_table=50)
print("Schema summary built (chars):", len(SCHEMA_SUMMARY))


Evaluation note: use the shared `nl2sql.eval` harness so VA/EM/EX are computed identically for every run.


**Seed note (important):**
- A `seed` controls exemplar sampling when `k > 0`.
- Different seeds mean different few-shot exemplar sets from the same pool.
- For `k=0` there are no exemplars, so seed has no practical effect.

**Experiment knobs in next cell:**
- `K_VALUES` + `SEEDS` for k/seed sweeps.
- `EXEMPLAR_STRATEGY` for exemplar-pool ablation (`all`, `brief_sql`, `join_heavy`, `agg_heavy`).
- `PROMPT_VARIANT` for prompt ablation (`default`, `schema_only_minimal`, `no_routing_hints`).
- `SCHEMA_VARIANT` for schema-context ablation (`full`, `first_80_lines`, `first_40_lines`).


In [ ]:
import re
import subprocess
import shutil
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd

from nl2sql.eval import eval_run
import nl2sql.prompting as prompting_mod

DEFAULT_SYSTEM_INSTRUCTIONS = prompting_mod.SYSTEM_INSTRUCTIONS

PROMPT_VARIANTS = {
    "default": DEFAULT_SYSTEM_INSTRUCTIONS,
    "schema_only_minimal": """You are an expert data analyst writing MySQL queries.
Given the database schema and a natural language question, write a single SQL SELECT query.

Rules:
- Output ONLY SQL (no explanation, no markdown).
- Output exactly ONE statement, starting with SELECT.
- Use only tables/columns listed in the schema.
""",
    "no_routing_hints": DEFAULT_SYSTEM_INSTRUCTIONS.split("- Routing hints:")[0].rstrip(),
}

def schema_variant_text(schema_text: str, variant: str) -> str:
    lines = schema_text.splitlines()
    if variant == "full":
        return schema_text
    if variant == "first_80_lines":
        return "\n".join(lines[:80])
    if variant == "first_40_lines":
        return "\n".join(lines[:40])
    raise ValueError(f"Unknown SCHEMA_VARIANT: {variant}")

def exemplar_pool_for_strategy(items: list[dict], strategy: str) -> list[dict]:
    if strategy == "all":
        return list(items)

    def _sql(x):
        return str(x.get("sql", "")).strip()

    def _is_join(sql: str) -> bool:
        s = sql.lower()
        return " join " in f" {s} "

    def _is_agg(sql: str) -> bool:
        return bool(re.search(r"\b(sum|avg|count|min|max)\s*\(", sql.lower()))

    if strategy == "brief_sql":
        ranked = sorted(items, key=lambda x: len(_sql(x)))
        keep = max(50, int(0.4 * len(ranked)))
        pool = ranked[:keep]
    elif strategy == "join_heavy":
        pool = [x for x in items if _is_join(_sql(x))]
    elif strategy == "agg_heavy":
        pool = [x for x in items if _is_agg(_sql(x))]
    else:
        raise ValueError(f"Unknown EXEMPLAR_STRATEGY: {strategy}")

    return pool if len(pool) >= 10 else list(items)

try:
    commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"]).decode().strip()
except Exception:
    commit = "unknown"

run_metadata_base = {
    "commit": commit,
    "model_id": MODEL_ID,
    "notebook": "02_baseline_prompting_eval.ipynb",
    "method": "baseline",
}

Path("results/baseline").mkdir(parents=True, exist_ok=True)

def run_baseline_grid(
    *,
    k_values: list[int],
    seeds: list[int],
    run_tag: str,
    prompt_variant: str,
    schema_variant: str,
    exemplar_strategy: str,
    limit: int | None = None,
    copy_canonical: bool = True,
):
    if not seeds:
        raise ValueError("Provide at least one seed")

    if prompt_variant not in PROMPT_VARIANTS:
        raise ValueError(f"Unknown PROMPT_VARIANT: {prompt_variant}")

    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%SZ")
    run_dir = Path("results/baseline/runs") / f"{run_tag}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=True)

    schema_used = schema_variant_text(SCHEMA_SUMMARY, schema_variant)
    exemplar_pool = exemplar_pool_for_strategy(test_set, exemplar_strategy)

    rows = []
    primary_seed = seeds[0]

    old_prompt = prompting_mod.SYSTEM_INSTRUCTIONS
    prompting_mod.SYSTEM_INSTRUCTIONS = PROMPT_VARIANTS[prompt_variant]

    try:
        for k in k_values:
            seed_list = [primary_seed] if k == 0 else seeds
            for seed in seed_list:
                save_path = run_dir / f"results_k{k}_seed{seed}.json"

                run_meta = dict(run_metadata_base)
                run_meta.update({
                    "run_tag": run_tag,
                    "k": k,
                    "seed": seed,
                    "prompt_variant": prompt_variant,
                    "schema_variant": schema_variant,
                    "exemplar_strategy": exemplar_strategy,
                    "exemplar_pool_size": len(exemplar_pool),
                })

                items = eval_run(
                    test_set=test_set,
                    exemplar_pool=exemplar_pool,
                    k=k,
                    limit=limit,
                    seed=seed,
                    engine=engine,
                    model=model,
                    tokenizer=tok,
                    schema_summary=schema_used,
                    save_path=str(save_path),
                    run_metadata=run_meta,
                    avoid_exemplar_leakage=True,
                )

                n = len(items)
                va = sum(int(x.va) for x in items) / max(n, 1)
                em = sum(int(x.em) for x in items) / max(n, 1)
                ex = sum(int(x.ex) for x in items) / max(n, 1)

                rows.append({
                    "run_tag": run_tag,
                    "prompt_variant": prompt_variant,
                    "schema_variant": schema_variant,
                    "exemplar_strategy": exemplar_strategy,
                    "exemplar_pool_size": len(exemplar_pool),
                    "k": k,
                    "seed": seed,
                    "n": n,
                    "va_rate": va,
                    "em_rate": em,
                    "ex_rate": ex,
                    "json_path": str(save_path),
                })

                if copy_canonical and seed == primary_seed and k in {0, 3}:
                    target = (
                        Path("results/baseline/results_zero_shot_200.json")
                        if k == 0
                        else Path("results/baseline/results_few_shot_k3_200.json")
                    )
                    target.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(save_path, target)
                    print(f"Updated canonical file: {target}")
    finally:
        prompting_mod.SYSTEM_INSTRUCTIONS = old_prompt

    df = pd.DataFrame(rows).sort_values(["k", "seed"]).reset_index(drop=True)
    df.to_csv(run_dir / "grid_summary.csv", index=False)

    agg = (
        df.groupby(["prompt_variant", "schema_variant", "exemplar_strategy", "k"], as_index=False)
        .agg(
            runs=("seed", "count"),
            va_mean=("va_rate", "mean"),
            va_std=("va_rate", "std"),
            em_mean=("em_rate", "mean"),
            em_std=("em_rate", "std"),
            ex_mean=("ex_rate", "mean"),
            ex_std=("ex_rate", "std"),
        )
    )
    agg.to_csv(run_dir / "grid_summary_by_k.csv", index=False)

    print("Saved grid run to:", run_dir)
    return df, agg, run_dir

# ============================
# QUICK MODE (recommended default)
K_VALUES = [0, 3]
SEEDS = [7]
RUN_TAG = "baseline_main"
PROMPT_VARIANT = "default"
SCHEMA_VARIANT = "full"
EXEMPLAR_STRATEGY = "all"

# FULL K/S E1 SWEEP (uncomment for primary experiment)
# K_VALUES = [0, 1, 3, 5, 8]
# SEEDS = [7, 17, 27, 37, 47]
# RUN_TAG = "baseline_e1_k_sweep"
# PROMPT_VARIANT = "default"
# SCHEMA_VARIANT = "full"
# EXEMPLAR_STRATEGY = "all"

# EXEMPLAR STRATEGY ABLATION EXAMPLE (few-shot only; keep k>0 values)
# K_VALUES = [3]
# SEEDS = [7, 17, 27, 37, 47]
# RUN_TAG = "baseline_exemplar_brief"
# EXEMPLAR_STRATEGY = "brief_sql"

# PROMPT/SCHEMA ABLATION EXAMPLE
# K_VALUES = [0, 3]
# SEEDS = [7]
# RUN_TAG = "baseline_prompt_ablation_schema_only"
# PROMPT_VARIANT = "schema_only_minimal"
# SCHEMA_VARIANT = "first_80_lines"

baseline_grid, baseline_by_k, baseline_run_dir = run_baseline_grid(
    k_values=K_VALUES,
    seeds=SEEDS,
    run_tag=RUN_TAG,
    prompt_variant=PROMPT_VARIANT,
    schema_variant=SCHEMA_VARIANT,
    exemplar_strategy=EXEMPLAR_STRATEGY,
    limit=None,
)

print("\nPer-run rows:")
display(baseline_grid)
print("\nPer-k summary (mean/std across seeds):")
display(baseline_by_k)


In [ ]:
# Quick summary (reads the saved JSON outputs)
import json

zero = json.loads(open("results/baseline/results_zero_shot_200.json", "r", encoding="utf-8").read())
few  = json.loads(open("results/baseline/results_few_shot_k3_200.json", "r", encoding="utf-8").read())

print("Zero-shot:", "VA", round(zero["va_rate"], 3), "EM", round(zero.get("em_rate", 0.0), 3), "EX", round(zero["ex_rate"], 3))
print("Few-shot:",  "VA", round(few["va_rate"], 3),  "EM", round(few.get("em_rate", 0.0), 3),  "EX", round(few["ex_rate"], 3))
